In [1]:
import torch 

Ref. V. Dumoulin and F. Visin, <i>A guide to convolution arithmetic for deep learning</i> <b>2016</b>. doi: 10.48550/ARXIV.1603.07285.

In [8]:
# Generación de un indice invertido. Cada línea en el indice representa
# una palabra (vector semantico)
# 
# La variable padding_idx es reservada como comodin para el caso de 
# que esta sea utilizada como relleno.
embedding = torch.nn.Embedding(10, 5, padding_idx=9)
embedding.weight

Parameter containing:
tensor([[ 0.5619, -0.7912, -1.9860,  0.4270, -1.6669],
        [-0.7633, -1.2504,  0.8393, -0.4949,  0.1892],
        [-0.8731, -0.7966, -0.5558,  0.1839, -1.2145],
        [-0.4237,  0.2841,  1.3995, -0.6623,  0.8048],
        [-0.5552, -0.9208, -1.4534, -0.8995,  0.8469],
        [-0.3159, -0.7477, -1.0075,  0.0575, -0.3975],
        [-0.9175,  0.3233, -0.8610,  1.1140,  2.1137],
        [ 0.9275, -0.4747, -0.2126,  0.8270, -0.5752],
        [ 0.6023,  0.4754, -0.6525,  0.5913,  1.3907],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], requires_grad=True)

**Ejemplo de un tensor 1D**

In [15]:
# Suponga una secuencia de palabras donde cada palabra es reemplazada por un 
# índice entero. Sea [1,2,3] tal secuencia. Cada índice de la secuencia es 
# transformado a su equivalente vectorial. 
sequence = embedding(torch.tensor([1, 2, 3]))

# Aplicación de la transpuesta a la secuencia
sequence = torch.transpose(embedding(torch.tensor([1, 2, 3])),0,1)
sequence

tensor([[-0.7633, -0.8731, -0.4237],
        [-1.2504, -0.7966,  0.2841],
        [ 0.8393, -0.5558,  1.3995],
        [-0.4949,  0.1839, -0.6623],
        [ 0.1892, -1.2145,  0.8048]], grad_fn=<TransposeBackward0>)

In [30]:
# 1-D tensor + convolución

T_1D = torch.tensor([1, 2, 3, 4, 5, 6], dtype = torch.float)
print(T_1D.shape)

# El tensor debe ser transformado a una matriz donde:
# - la primera columna corresponda al número de canales de entrada
# - la segunda columna corresponda al número de canales de salida
# - la tercera columna corresponda al tamaño de la señal 
T_1D = T_1D.unsqueeze(0).unsqueeze(0)
print(T_1D.shape)

# En este ejemplo, aplico un núcleo de tamaño 3. Hize que el número de salidas
# sea igual a 1. Si el número de canales de salida es 3, internamente se crean 
# tres kernels los cuales se aplican durante la fase de convolución.
conv = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1,  bias=False)

# Para propósito de debug. Inicializo la matriz cero a unos. Si el número de 
# canales es mayor que 1. Entocnes es necesario inicializar las matrices de 
# coeficientes adicionales que se crean.
with torch.no_grad():
    conv.weight[0,:,:] = 1.

# Se imprime los coeficientes de la primera matriz.
print("The weight matrix weights are ", conv.weight)

# Se aplica la convolución y se imprime los coeficientes resultantes.
print("Result : ", conv(T_1D))

# Every thing worked as expected.

torch.Size([6])
torch.Size([1, 1, 6])
The weight matrix weights are  Parameter containing:
tensor([[[1., 1., 1.]]], requires_grad=True)
Result :  tensor([[[ 6.,  9., 12., 15.]]], grad_fn=<SqueezeBackward1>)


In [53]:
# 2-D tensor + convolution
T_2D = torch.tensor([
    [1,1,1,1,1],
    [2,2,2,2,2],
    [3,3,3,3,3],
    [4,4,4,4,4]
], dtype = torch.float)


# El tensor debe ser transformado a una matriz donde:
# - la primera columna corresponda al número de canales de entrada (tamaño del embedding)
# - la segunda columna corresponda al número de canales de salida (el número de núcleos)
# - la tercera columna corresponda al tamaño de la señal 
T_2D = T_2D.unsqueeze(0)
print("El contenido del tensor es : ", T_2D)
print("La dimensión del tensor es : ", T_2D.shape)

# En este ejemplo. Cada canal representa una palabra codificada en su forma vectorial
# Deseo un kernel bidimensional unitario con un stride de 1 
conv = torch.nn.Conv1d(in_channels=5, out_channels=1, kernel_size=5, stride=1,  bias=False)

# Para propósito de debug. Inicializo la matriz cero a unos. Si el número de 
# canales es mayor que 1. Entocnes es necesario inicializar las matrices de 
# coeficientes adicionales que se crean.
with torch.no_grad():
    conv.weight[:,:,:] = 1.

# Se imprime los coeficientes de la primera matriz.
print("The weight matrix weights are ", conv.weight)

# Se aplica la convolución y se imprime los coeficientes resultantes.
print("Result : ", conv(T_2D))

El contenido del tensor es :  tensor([[[1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2.],
         [3., 3., 3., 3., 3.],
         [4., 4., 4., 4., 4.]]])
La dimensión del tensor es :  torch.Size([1, 4, 5])
The weight matrix weights are  Parameter containing:
tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]], requires_grad=True)


RuntimeError: Given groups=1, weight of size 1 5 5, expected input[1, 4, 5] to have 5 channels, but got 4 channels instead

In [7]:
# 2-D tensor, 3 channels
T_2D = torch.tensor([  
    [[1,1,0,1,1],[2,2,0,2,2],[3,3,0,3,3],[4,4,0,4,4],[5,5,0,5,5]], 
    [[1,0,1,0,1],[2,0,2,0,2],[3,0,3,0,3],[4,0,4,0,4],[5,0,5,0,5]],
    [[0,1,0,1,0],[0,2,0,2,0],[0,3,0,3,0],[0,4,0,4,0],[0,5,0,5,0]]
], dtype = torch.float)
T_2D

tensor([[[1., 1., 0., 1., 1.],
         [2., 2., 0., 2., 2.],
         [3., 3., 0., 3., 3.],
         [4., 4., 0., 4., 4.],
         [5., 5., 0., 5., 5.]],

        [[1., 0., 1., 0., 1.],
         [2., 0., 2., 0., 2.],
         [3., 0., 3., 0., 3.],
         [4., 0., 4., 0., 4.],
         [5., 0., 5., 0., 5.]],

        [[0., 1., 0., 1., 0.],
         [0., 2., 0., 2., 0.],
         [0., 3., 0., 3., 0.],
         [0., 4., 0., 4., 0.],
         [0., 5., 0., 5., 0.]]])